<a href="https://colab.research.google.com/github/lesley2023/MM/blob/main/a1_code_q1b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install numba

In [ ]:
import numpy as np
import scipy.stats as sts
import time
import timeit
from numba.pycc import CC

<ipython-input-2-df8fab950686>:5: NumbaPendingDeprecationWarning: The 'pycc' module is pending deprecation. Replacement technology is being developed.

Pending Deprecation in Numba 0.57.0. For more information please see: https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-the-numba-pycc-module
  from numba.pycc import CC


In [ ]:
# Define a function to inspect variable types within the loop
def inspect_variable_types(rho, mu, sigma, z_0, S, T):
    np.random.seed(25)
    eps_mat = np.random.normal(loc=0, scale=sigma, size=(T, S))
    z_mat = np.zeros((T, S))

    variable_types = {}


    if S > 0 and T > 0:
        s_ind = 0
        z_tm1 = z_0
        t_ind = 0
        e_t = eps_mat[t_ind, s_ind]
        z_t = rho * z_tm1 + (1 - rho) * mu + e_t

        # Collecting types
        variable_types['rho'] = type(rho)
        variable_types['mu'] = type(mu)
        variable_types['sigma'] = type(sigma)
        variable_types['z_0'] = type(z_0)
        variable_types['eps_mat'] = type(eps_mat)
        variable_types['z_mat'] = type(z_mat)
        variable_types['s_ind'] = type(s_ind)
        variable_types['z_tm1'] = type(z_tm1)
        variable_types['t_ind'] = type(t_ind)
        variable_types['e_t'] = type(e_t)
        variable_types['z_t'] = type(z_t)

    return variable_types

In [ ]:
# Example usage with some arbitrary parameters
variable_types = inspect_variable_types(0.9, 0.1, 0.2, 1.0, 10, 10)
for variable, var_type in variable_types.items():
    print(f"{variable}: {var_type}")

rho: <class 'float'>
mu: <class 'float'>
sigma: <class 'float'>
z_0: <class 'float'>
eps_mat: <class 'numpy.ndarray'>
z_mat: <class 'numpy.ndarray'>
s_ind: <class 'int'>
z_tm1: <class 'float'>
t_ind: <class 'int'>
e_t: <class 'numpy.float64'>
z_t: <class 'numpy.float64'>


In [ ]:
cc = CC('test_aot')

@cc.export('loop_aot', 'f8[:,:](f8,f8,f8,f8,i4,i4)')
def loop_aot(rho, mu, sigma, z_0, S, T):
    np.random.seed(25)
    eps_mat = np.random.normal(loc=0, scale=sigma, size=(T, S))
    z_mat = np.zeros((T, S))
    for s_ind in range(S):
        z_tm1 = z_0
        for t_ind in range(T):
            e_t = eps_mat[t_ind, s_ind]
            z_t = rho * z_tm1 + (1 - rho) * mu + e_t
            z_mat[t_ind, s_ind] = z_t
            z_tm1 = z_t
    return z_mat

cc.compile()
import test_aot

In [ ]:
rho = 0.5
mu = 3.0
sigma = 1.0
z_0 = mu
S = 1000
T = 4160

start_time = time.time()

# Simulate with original Python code
np.random.seed(25)
eps_mat = sts.norm.rvs(loc=0, scale=sigma, size=(T, S))
z_mat = np.zeros((T, S))
for s_ind in range(S):
    z_tm1 = z_0
    for t_ind in range(T):
        e_t = eps_mat[t_ind, s_ind]
        z_t = rho * z_tm1 + (1 - rho) * mu + e_t
        z_mat[t_ind, s_ind] = z_t
        z_tm1 = z_t

total_time_original = time.time() - start_time

In [ ]:
timing_globals = {
    "rho": rho,
    "mu": mu,
    "sigma": sigma,
    "z_0": z_0,
    "S": S,
    "T": T,
    "loop_aot": test_aot.loop_aot
}

In [ ]:
# Timing the AOT compiled version
aot_time = timeit.timeit("loop_aot(rho, mu, sigma, z_0, S, T)", globals=timing_globals, number=10)

# Calculating and displaying the speedup
speedup_factor = total_time_original / aot_time
print(f"Original code time: {total_time_original:.2f} seconds")
print(f"AOT compiled code time: {aot_time:.2f} seconds")
print(f"AOT compiled version is {speedup_factor:.2f} times faster.")


Original code time: 5.20 seconds
AOT compiled code time: 2.38 seconds
AOT compiled version is 2.19 times faster.
